In [1]:
import numpy as np
import pandas as pd

In [ ]:
#first, correct cancer files read count biases

In [2]:
dilution_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name='test')
dilution_calc

,healthy_file_name,healthy_read_counts,cancer_file_label,cancer_file_name,original,cancer_read_counts,correction_factor,corrected read counts
0,WHC567,1700904906,CRC-9,1279_221015,0.455,2295293721,0.741040,1.700905e+09
1,WHC567,1700904906,CRC-10,1279_241116,0.858,2331916340,0.729402,1.700905e+09
2,WHC567,1700904906,CRC-11,512_051015,0.490,2297120422,0.740451,1.700905e+09
3,WHC567,1700904906,CRC-12,512_130114,0.351,2273365787,0.748188,1.700905e+09
4,WHC568,1638484969,CRC-9,1279_221015,0.455,2295293721,0.713845,1.638485e+09
...,...,...,...,...,...,...,...,...
67,WHC565,1576440271,CRC-12,512_130114,0.351,2273365787,0.693439,1.576440e+09
68,WHC566,1698345718,CRC-9,1279_221015,0.455,2295293721,0.739925,1.698346e+09
69,WHC566,1698345718,CRC-10,1279_241116,0.858,2331916340,0.728305,1.698346e+09
70,WHC566,1698345718,CRC-11,512_051015,0.490,2297120422,0.739337,1.698346e+09


In [3]:
num_list_h = ['567', '568', '569', '570', '571', '572']
num_list_c = ['1279_221015', '1279_241116', '512_051015', '512_130114' ]

In [4]:
with open('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\benchmark_region_healthy_may30') as file:
    regions_h = [line.rstrip() for line in file]
len(regions_h)

6

In [5]:
regions_h

['chr1_156028951-156032951',
 'chr19_838960-842960',
 'chr19_35737922-35741922',
 'chr19_41080891-41084891',
 'chr4_185745070-185749070',
 'chr16_75283369-75287369']

In [6]:
regions_c = [region[3:] for region in regions_h]
len(regions_c)

6

In [7]:
regions_c

['1_156028951-156032951',
 '19_838960-842960',
 '19_35737922-35741922',
 '19_41080891-41084891',
 '4_185745070-185749070',
 '16_75283369-75287369']

In [8]:
for num_c in num_list_c:
    for num_h in num_list_h:
        for region in regions_c:
            fn = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_test\\may30-benchmark-raw-cancer-' + \
                 num_c + '-' + region
            df = pd.read_csv(fn, header=None, sep='\t')
            factor = dilution_calc[(dilution_calc.healthy_file_name == 'WHC' + num_h) & \
                              (dilution_calc.cancer_file_name == num_c)]['correction_factor'].values[0]
            df[2] = df[2] * factor
            df.to_csv('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_test\\' + num_c + \
                      '\\WHC' + num_h + '\\benchmark-' + region)

In [7]:
#second, mix!

In [9]:
file_origin_column = []
fraction_column = []
regions_dic = {}
for region_c in regions_c:
    region_h = 'chr' + region_c
    region_column = []
    for num_c in num_list_c:
        for num_h in num_list_h:
            frac_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name = num_c)
            fn_c = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_test\\' + num_c + \
                    '\\WHC' + num_h + '\\benchmark-' + region_c
            fn_h = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\healthy_raw_test\\may30-benchmark-raw-healthy-WHC' + \
                    num_h + '-' + region_h
            raw_c = pd.read_csv(fn_c)['2']
            raw_h = pd.read_csv(fn_h, header = None, sep = '\t')[2]
            for fraction in frac_calc['target_fraction']:
                if len(regions_dic.keys()) == 0:
                    file_origin_column.append(num_c + num_h)
                    fraction_column.append(fraction)
                
                healthy_factor = frac_calc[frac_calc.target_fraction == fraction]['healthy_factor'].values[0]
                cancer_factor = frac_calc[frac_calc.target_fraction == fraction]['cancer_factor'].values[0]
                
                raw_mixed = cancer_factor * raw_c + healthy_factor * raw_h
                
                if region_h in ['chr1_156028951-156032951', 'chr19_838960-842960', 'chr19_35737922-35741922']:
                    left_flank_sum = raw_mixed[0:1000].sum()
                    right_flank_sum = raw_mixed[3000:4000].sum()
                    flank_mean = (left_flank_sum + right_flank_sum) / 2000
                    avg = (raw_mixed[1850:2050] / flank_mean).mean()
                elif region_h in ['chr4_185745070-185749070', 'chr16_75283369-75287369']:
                    left_flank_sum = raw_mixed[3000:4000].sum()
                    right_flank_sum = raw_mixed[0:1000].sum()
                    flank_mean = (left_flank_sum + right_flank_sum) / 2000
                    avg = (raw_mixed[2100:2300] / flank_mean).mean()                    
                else:
                    left_flank_sum = raw_mixed[0:1000].sum()
                    right_flank_sum = raw_mixed[3000:4000].sum()
                    flank_mean = (left_flank_sum + right_flank_sum) / 2000
                    avg = (raw_mixed[1700:1900] / flank_mean).mean()
                
                region_column.append(avg)
    regions_dic[region_c] = region_column

In [10]:
regions_df = pd.DataFrame(regions_dic)
regions_df['fraction'] = fraction_column
regions_df['file_origin'] = file_origin_column
regions_df

,1_156028951-156032951,19_838960-842960,19_35737922-35741922,19_41080891-41084891,4_185745070-185749070,16_75283369-75287369,fraction,file_origin
0,0.817436,1.284408,0.987339,0.457071,0.165877,0.598541,0.000,1279_221015567
1,0.815805,1.284829,0.985499,0.457751,0.167848,0.599822,0.005,1279_221015567
2,0.814170,1.285253,0.983657,0.458429,0.169828,0.601113,0.010,1279_221015567
3,0.812531,1.285681,0.981814,0.459105,0.171817,0.602415,0.015,1279_221015567
4,0.810888,1.286111,0.979970,0.459779,0.173814,0.603727,0.020,1279_221015567
...,...,...,...,...,...,...,...,...
697,0.790787,1.171080,0.930378,0.403486,0.229104,0.559976,0.300,512_130114572
698,0.766478,1.208249,0.883794,0.413806,0.237465,0.532307,0.325,512_130114572
699,0.740663,1.246864,0.833704,0.424742,0.246240,0.502804,0.350,512_130114572
700,0.713198,1.287010,0.779696,0.436352,0.255461,0.471280,0.375,512_130114572


In [11]:
regions_df.to_csv('test_benchmark.csv')